In [12]:
import pandas as pd
import numpy as np
import requests
import json
from census import Census
from us import states


from config import covidkey
from config import censuskey
c = Census(censuskey, year=2013)
censuskey


'04f514dfbd7422da53425fd0190e7284a2214b9d'

In [2]:
census_url=f"api.census.gov/data/2019/pep/population?get=COUNTY,DATE_CODE,DATE_DESC,DENSITY,POP,NAME,STATE&for=region:*&key={censuskey}"

census_url

'api.census.gov/data/2019/pep/population?get=COUNTY,DATE_CODE,DATE_DESC,DENSITY,POP,NAME,STATE&for=region:*&key=04f514dfbd7422da53425fd0190e7284a2214b9d'

In [3]:
covidnow_st_df=pd.read_csv("CSVs/Covid_Now_CSVs/states.csv")

covidnow_st_df.head()

,fips,country,state,county,level,lat,locationId,long,population,metrics.testPositivityRatio,...,actuals.newCases,actuals.vaccinesDistributed,actuals.vaccinationsInitiated,actuals.vaccinationsCompleted,lastUpdatedDate,url,metrics.vaccinationsInitiatedRatio,metrics.vaccinationsCompletedRatio,actuals.newDeaths,actuals.vaccinesAdministered
0,2,US,AK,NaN,state,NaN,iso1:us#iso2:us-ak,NaN,731545,0.066,...,439,848165,372132,330406,2021-07-24,https://covidactnow.org/us/alaska-ak,0.509,0.452,2,690518
1,1,US,AL,NaN,state,NaN,iso1:us#iso2:us-al,NaN,4903185,0.155,...,1733,4979270,2041597,1663031,2021-07-24,https://covidactnow.org/us/alabama-al,0.416,0.339,4,3494892
2,5,US,AR,NaN,state,NaN,iso1:us#iso2:us-ar,NaN,3017804,0.102,...,1987,2937300,1352937,1077347,2021-07-24,https://covidactnow.org/us/arkansas-ar,0.448,0.357,6,2363840
3,4,US,AZ,NaN,state,NaN,iso1:us#iso2:us-az,NaN,7278717,0.106,...,1479,8399110,3798435,3259542,2021-07-24,https://covidactnow.org/us/arizona-az,0.522,0.448,7,7001414
4,6,US,CA,NaN,state,NaN,iso1:us#iso2:us-ca,NaN,39512223,0.046,...,8933,50392775,25192441,20594557,2021-07-24,https://covidactnow.org/us/california-ca,0.638,0.521,45,44467248


In [4]:
# covidnow_cou_df = ('CSVs/Covid_Now_CSVs/counties.csv')

# covidnow_cou_df

In [6]:
oxford_df=pd.read_csv("CSVs/OxCGRT_US_latest.csv")

oxford_df.head()

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C1_Notes,C2_Workplace closing,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,United States,USA,NaN,NaN,NAT_GOV,20200101,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,United States,USA,NaN,NaN,NAT_GOV,20200102,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,United States,USA,NaN,NaN,NAT_GOV,20200103,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,United States,USA,NaN,NaN,NAT_GOV,20200104,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,United States,USA,NaN,NaN,NAT_GOV,20200105,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
census_data = c.acs5.get('B01003_001E', {'for': 'state:*'})
census_pd = pd.DataFrame(census_data)
census_pd = census_pd.rename(columns ={'B01003_001E': 'Population'})
census_pd.head()

,Population,state
0,4799277.0,01
1,720316.0,02
2,6479703.0,04
3,2933369.0,05
4,37659181.0,06


In [44]:
census_pd['State'] = " "
census_pd.head()

,Population,state,State
0,4799277.0,01,
1,720316.0,02,
2,6479703.0,04,
3,2933369.0,05,
4,37659181.0,06,


In [50]:
states_dic = []
for x in census_pd['state']:
    states_dic = states_dic.append(states.lookup(x).abbr)
#    census_pd['State'].append(states.lookup(x).abbr)
#    census_pd['State'] = states.lookup(x).abbr
#    census_pd['State'][x] = states.lookup(census_pd['state'][x]).abbr
    
# State_list = census_pd['State'].tolist()
# State_list
states_dic

AttributeError: 'NoneType' object has no attribute 'append'

In [38]:
census_pd

,Population,state,State
0,4799277.0,01,PR
1,720316.0,02,PR
2,6479703.0,04,PR
3,2933369.0,05,PR
4,37659181.0,06,PR
5,5119329.0,08,PR
6,908446.0,10,PR
7,619371.0,11,PR
8,3583561.0,09,PR
9,19091156.0,12,PR
